# Timesketch Sigma rule from file to Database notebook

This notebook will help adding one Sigma rule from disk to the Timesketch Sigma rule database.

In [5]:
!pip install -q timesketch_api_client

In [1]:
!pip uninstall -y timesketch_api_client

In [1]:
import sys

MODULE_FULL_PATH = '/usr/local/google/home/jaegeral/dev/timesketch/api_client/python/'

sys.path.insert(1, MODULE_FULL_PATH)

In [2]:
!python3 /usr/local/google/home/jaegeral/dev/timesketch/api_client/python/setup.py build && python3 /usr/local/google/home/jaegeral/dev/timesketch/api_client/python/setup.py install --user

running build
running install
/usr/local/google/home/jaegeral/dev/timesketch/.venv/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/google/home/jaegeral/dev/timesketch/.venv/lib/python3.10/site-packages/setuptools/command/easy_install.py:158: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
Checking .pth file support in /usr/local/google/home/jaegeral/.local/lib/python3.10/site-packages/
/usr/local/google/home/jaegeral/dev/timesketch/.venv/bin/python3 -E -c pass
TEST FAILED: /usr/local/google/home/jaegeral/.local/lib/python3.10/site-packages/ does NOT support .pth files
bad install directory or PYTHONPATH

You are attempting to install a package to a directory that is not
on PYTHONPATH and which Python does not read ".pth" files from.  The
installat

In [6]:
import json

from timesketch_api_client import config
from timesketch_api_client import sketch
from timesketch_api_client import timeline
from timesketch_api_client import analyzer
from timesketch_api_client import search

from timesketch_import_client import helper
from timesketch_import_client import importer

If an error occurs, you might need to remove `rm ~/.timesketch*` and re-run the following command

In [7]:
ts_client = config.get_client()

In [32]:
sketches = [(x.id, x.name) for x in ts_client.list_sketches()]
for sketch in sketches:
    print(f"Sketch name: {sketch[1]} ({sketch[0]})")

Sketch name: aaa (1)


Using sketch ID 1
{
    "meta": {
        "aggregators": {
            "field_bucket": {
                "description": "Aggregating values of a particular field",
                "display_name": "Terms Aggregation",
                "form_fields": [
                    {
                        "display": true,
                        "label": "Chart type to render",
                        "name": "supported_charts",
                        "options": [
                            "table",
                            "hbarchart",
                            "barchart",
                            "linechart",
                            "circlechart"
                        ],
                        "type": "ts-dynamic-form-select-input"
                    },
                    {
                        "default_value": "",
                        "display": true,
                        "label": "What field to aggregate on",
                        "name": "field",
               

# Get Sigma rules

In [9]:
ts_client.version

'API Client: 20221005\nTS Backend: 20210602'

In [16]:
sigme_rule = ts_client.list_sigmarules(as_pandas=True)

Deprecated, please use list_sigmarules() instead


In [17]:
sigme_rule

,author,created_at,date,description,detection,falsepositives,file_name,id,level,logsource,modified,references,rule_uuid,rule_yaml,search_query,status,tags,title,updated_at
0,None,2022-10-04 15:10:27.697184,2022/10/04,Detects suspicious FOOBAR,"{'condition': 'keywords', 'keywords': ['*fooba...",[Unknown],N/A,7f4d2d41-19f9-4427-9227-1ef288247f52,medium,"{'product': 'google_workspace', 'service': 'go...",2022/10/04,[https://],7f4d2d41-19f9-4427-9227-1ef288247f52,title: Foobar\nid: 7f4d2d41-19f9-4427-9227-1ef...,*foobar2*,experimental,[None],Foobar,2022-10-04 16:40:49.916821
1,None,2022-10-04 16:48:25.620548,2022/10/04,Detects suspicious FOOBAR,"{'condition': 'keywords', 'keywords': ['*fooba...",[Unknown],N/A,f71185d4-f2a2-4054-85b3-94c3ac9ce438,medium,"{'category': 'process_creation', 'product': 'l...",2022/10/04,[https://],f71185d4-f2a2-4054-85b3-94c3ac9ce438,title: Foobar\nid: f71185d4-f2a2-4054-85b3-94c...,"(data_type:(""shell:zsh:history"" OR ""bash:histo...",experimental,[None],Foobar,2022-10-04 16:48:25.620548
2,None,2022-10-04 16:49:04.403246,2022/10/04,Detects suspicious FOOBAR,"{'condition': 'keywords', 'keywords': ['*zmap*']}",[Unknown],N/A,1289ad6d-1801-44aa-94f6-e16e596028b5,medium,"{'category': 'clamav', 'product': 'linux'}",2022/10/04,[https://],1289ad6d-1801-44aa-94f6-e16e596028b5,title: Foobar\nid: 1289ad6d-1801-44aa-94f6-e16...,*zmap*,experimental,[None],Foobar,2022-10-04 16:49:04.403246


In [20]:
new_rule = '''
title: Installation of foobar
id: bb1e0d1d-cd13-4b65-bf7e-69b4e740266ba
description: Detects suspicious installation of foobar
references:
    - https://samle.com/foobar
author: Alexander Jaeger
date: 2020/12/10
modified: 2020/12/10
logsource:
    product: linux
    service: shell
detection:
    keywords:
        # Generic suspicious commands
        - '*apt-get install foobar*'
    condition: keywords
falsepositives:
    - Unknown
level: high
'''

In [21]:
result = ts_client.create_sigmarule(new_rule)

In [24]:
result.status

'experimental'

In [27]:
# Provide a path where a rule is stored

RULE_File = '/usr/local/google/home/jaegeral/dev/timesketch/data/sigma/rules/lnx_susp_zmap.yml'

with open(RULE_File, 'r') as sigma_rule_file:
        sigma_rule = sigma_rule_file.read()
        
#sigma_config = sigma_configuration.SigmaConfiguration(sigma_config)

In [26]:
# Check that the file is actually read
sigma_rule

"title: Suspicious Installation of ZMap\nid: 5266a592-b793-11ea-b3de-0242ac130004\ndescription: Detects suspicious installation of ZMap\nreferences:\n    - https://rmusser.net/docs/ATT&CK-Stuff/ATT&CK/Discovery.html\nauthor: Alexander Jaeger\ndate: 2020/06/26\nmodified: 2020/06/26\ntags:\n    - attack.discovery\n    - attack.t1046\nlogsource:\n    product: linux\n    service: shell\ndetection:\n    keywords:\n        # Generic suspicious commands\n        - '*apt-get install zmap*'\n    condition: keywords\nfalsepositives:\n    - Unknown\nlevel: high\n"

In [30]:
parsed_rule = ts_client.parse_sigmarule_by_text(sigma_rule)
parsed_rule.search_query

Deprecated, use SigmaRule object instead


'(data_type:("shell:zsh:history" OR "bash:history:command" OR "apt:history:line" OR "selinux:line") AND "apt-get install zmap")'

In [32]:
ts_client.create_sigmarule(sigma_rule)

Failed response: [403] Rule 5266a592-b793-11ea-b3de-0242ac130004 was already found in the database FORBIDDEN
Failed response: [403] Rule 5266a592-b793-11ea-b3de-0242ac130004 was already found in the database FORBIDDEN
Failed response: [403] Rule 5266a592-b793-11ea-b3de-0242ac130004 was already found in the database FORBIDDEN
Failed response: [403] Rule 5266a592-b793-11ea-b3de-0242ac130004 was already found in the database FORBIDDEN
Failed response: [403] Rule 5266a592-b793-11ea-b3de-0242ac130004 was already found in the database FORBIDDEN


RuntimeError: Unable to create a new Sigma Rule.

In [ ]:
# you can only add the same rule once:

ts_client.create_sigmarule(sigma_rule)

# will give you:
# Failed response: [403] Rule 5266a592-b793-11ea-b3de-0242ac130004 was already found in the database FORBIDDEN